In [2]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies.csv')
df = df.drop(columns=['judul_film'])

In [4]:
df.tail()

,ringkasan_sinopsis,genre
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis
1004,"Atika (Meriam Bellina) mantan penyanyi tenar, ...",Romantis


In [5]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [6]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [7]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [17]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(256, activation='selu'),
        tf.keras.layers.Dense(128, activation='selu'),
        tf.keras.layers.Dense(64, activation='selu'),
        tf.keras.layers.Dense(16, activation='selu'),
        tf.keras.layers.Dense(5, activation='softmax'),
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 6s - loss: 1.6199 - accuracy: 0.1990 - val_loss: 1.6116 - val_accuracy: 0.2139 - 6s/epoch - 212ms/step
Epoch 2/30
26/26 - 2s - loss: 1.6152 - accuracy: 0.1928 - val_loss: 1.6147 - val_accuracy: 0.1393 - 2s/epoch - 84ms/step
Epoch 3/30
26/26 - 2s - loss: 1.6143 - accuracy: 0.2127 - val_loss: 1.6079 - val_accuracy: 0.1940 - 2s/epoch - 84ms/step
Epoch 4/30
26/26 - 2s - loss: 1.6127 - accuracy: 0.1990 - val_loss: 1.6113 - val_accuracy: 0.2189 - 2s/epoch - 83ms/step
Epoch 5/30
26/26 - 2s - loss: 1.6099 - accuracy: 0.2077 - val_loss: 1.6149 - val_accuracy: 0.1443 - 2s/epoch - 86ms/step
Epoch 6/30
26/26 - 2s - loss: 1.6067 - accuracy: 0.2114 - val_loss: 1.6014 - val_accuracy: 0.2239 - 2s/epoch - 83ms/step
Epoch 7/30
26/26 - 2s - loss: 1.6018 - accuracy: 0.2488 - val_loss: 1.5946 - val_accuracy: 0.2189 - 2s/epoch - 85ms/step
Epoch 8/30
26/26 - 2s - loss: 1.4512 - accuracy: 0.3234 - val_loss: 1.5743 - val_accuracy: 0.2438 - 2s/epoch - 86ms/step
Epoch 9/30
26/26 - 2s - loss: 1